# Medoid Compositing
*Seasonal Composite Landsat TM/ETM+ Images Using the Medoid (a Multi-Dimensional Median), Neil Flood, 2013, doi:10.3390/rs5126481*

In [ ]:
import ee
ee.Initialize()

In [ ]:
from geetools import tools, composite, cloud_mask, indices

In [ ]:
import ipygee as ui

## Build a collection

In [ ]:
p = ee.Geometry.Point(-72, -42)

In [ ]:
col = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
        .filterBounds(p).filterDate('2017-01-01', '2017-12-01')\
        .map(cloud_mask.landsat8SRPixelQA())\
        .map(lambda img: img.addBands(indices.ndvi(img,'B5', 'B4')))\
        .limit(7)

In [ ]:
ui.eprint(col.size())

## Other simple composites to compare

In [ ]:
max_ndvi = col.qualityMosaic('ndvi')

In [ ]:
mosaic = col.mosaic()

## Medoid

In [ ]:
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']

### add date band before compositing

In [ ]:
def add_date(img):
    date = tools.date.getDateBand(img)
    return img.addBands(date).copyProperties(date, ['day_since_epoch'])
col = col.map(add_date)

In [ ]:
minval = ee.Number(col.aggregate_min('day_since_epoch'))

In [ ]:
maxval = ee.Number(col.aggregate_max('day_since_epoch'))

In [ ]:
medoid = composite.medoid(col, bands=bands)

In [ ]:
medoid = medoid.set('min_date', minval, 'max_date', maxval)

## Medoid without taking in count zero values

In [ ]:
medoid_no_zeros = composite.medoid(col, bands=bands, discard_zeros=True)

In [ ]:
medoid_no_zeros = medoid_no_zeros.set('min_date', minval, 'max_date', maxval)

## Show on Map

In [ ]:
Map = ui.Map()
Map.show()

In [ ]:
vis = {'bands':['B5', 'B6','B4'], 'min':0, 'max':5000}

In [ ]:
Map.addLayer(p)
Map.centerObject(p)

In [ ]:
Map.addLayer(max_ndvi, vis, 'max NDVI')

In [ ]:
Map.addLayer(mosaic, vis, 'simply Mosaic')

In [ ]:
Map.addLayer(medoid, vis, 'Medoid')

In [ ]:
Map.addLayer(medoid.select('date').randomVisualizer(), {'bands':['viz-red', 'viz-green', 'viz-blue'], 'min':0, 'max':255}, 'dates')

In [ ]:
Map.addLayer(medoid_no_zeros, vis, 'Medoid without zero values')

In [ ]:
Map.addLayer(medoid_no_zeros.select('date').randomVisualizer(), {'bands':['viz-red', 'viz-green', 'viz-blue'], 'min':0, 'max':255}, 'dates of medoid without zero values')

## Extract data from images and compute locally to compare

Extract medoid values in point

In [ ]:
medoid_values = tools.image.getValue(medoid.select(bands), p, scale=30, side='client')

In [ ]:
medoid_values

List of values

In [ ]:
medoid_values_list = [val for _, val in medoid_values.items()]

In [ ]:
medoid_values_list

Extract values at point in each image of the collection

In [ ]:
col_values = tools.imagecollection.getValues(col.select(bands), p, scale=30, side='client')

Get bandnames

In [ ]:
col_key_list = []
for _, d in col_values.items():
    keys = []
    for k, v in d.items():
        keys.append(k)        
    col_key_list.append(keys)

In [ ]:
col_key_list

Get values as a list

In [ ]:
col_values_list = []
for _, d in col_values.items():
    values = []
    for _, v in d.items():
        if v:
            values.append(v)
        else:
            values.append(0)
    col_values_list.append(values)

In [ ]:
col_values_list

## Medoid Method locally

In [ ]:
def local_medoid(values):
    from copy import copy
    import math

    def distance(arr1, arr2):
        zipped = zip(arr1, arr2)
        accum = 0
        for a, b in zipped:
            calc = (a-b)*(a-b)
            accum += calc
        return math.sqrt(accum)

    def med(values):
        results = {}
        for i, val in enumerate(values):
            val = list(val)
            cop = copy(values)
            cop = [list(a) for a in cop]
            cop.remove(val)
            dist = 0
            for r in cop:
                r = list(r)
                d = distance(val, r)
                dist += d
            results[i] = dist

        return results
    
    def getmin(d):
        minval = min(d.values())
        for k, v in d.items():
            if v == minval:
                return k
    
    values = med(values)
    min_value = getmin(values)
    
    # return the index of the minimized sum as first argument, and all options as second
    return min_value, values

## Compute medoid locally and compare

In [ ]:
local = local_medoid(col_values_list)

In [ ]:
local

Get the values that correspond to the medoid

In [ ]:
min_values = col_values_list[local[0]]

In [ ]:
min_values

Match bands with values

In [ ]:
local_medoid = dict(zip(col_key_list[0], min_values))

In [ ]:
local_medoid

In [ ]:
medoid_values

## Finally, compare values from medoid mosaic against locally computed medoid (from images values)

In [ ]:
medoid_values == local_medoid